# MongoDB

---

**Источники:**

[Большой туториал по MongoDB](https://medium.com/@Merrick_krg/%D0%B1%D0%BE%D0%BB%D1%8C%D1%88%D0%BE%D0%B9-%D1%82%D1%83%D1%82%D0%BE%D1%80%D0%B8%D0%B0%D0%BB-%D0%BF%D0%BE-mongodb-7e2f9e17f0c7)

[MongoDB vs MySQL: A Comparative Study on Databases](https://www.simform.com/mongodb-vs-mysql-databases)

[Academy 3T](https://studio3t.com/academy/)

[Robo 3T GUI](https://robomongo.org/download)

---

## Терминология

**MongoDB** — документоориентированная система управления базами данных, не требующая описания схемы таблиц.

Считается одним из классических примеров NoSQL-систем. 

Использует JSON-подобные документы и схему базы данных.

Написана на языке C++. 

<center><img src="images/MongoDB_database_colection.png"/></center>

<center><img src="images/rdbms_vs_mongodb.png"/></center>

<center><img src="images/mongodb_and_rdbms_schema.png"/></center>

**MongoDB подходит для следующих применений**:
- хранение и регистрация событий;
- системы управления документами и контентом;
- электронная коммерция;
- игры;
- данные мониторинга, датчиков;
- мобильные приложения;
- хранилище операционных данных веб-страниц (например, хранение комментариев, рейтингов, профилей пользователей, сеансы пользователей).

---

## Подготовка окружения

### Установка MongoDB

[MongoDB Community Server](https://www.mongodb.com/try/download/community)

[Robo 3T: the hobbyist GUI](https://robomongo.org/download)

In [1]:
!which pip

/home/ira/anaconda3/envs/LevelUp_DataScience/bin/pip


In [1]:
!conda install pymongo -y

Solving environment: done

## Package Plan ##

  environment location: /home/ira/anaconda3/envs/LevelUp_DataScience

  added / updated specs:
    - pymongo


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    pymongo-3.11.2             |   py38h2531618_0         1.2 MB
    ------------------------------------------------------------
                                           Total:         1.2 MB

The following NEW packages will be INSTALLED:

  pymongo            pkgs/main/linux-64::pymongo-3.11.2-py38h2531618_0



pymongo-3.11.2       | 1.2 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


---

## MongoDB Типы данных


---

## MongoDB + Python

In [5]:
import pymongo
from pymongo import MongoClient

pymongo.__version__

'3.11.2'

---

### Подключение к базе данных

In [21]:
# данные для подключения к БД
host='localhost'
port=3306

In [22]:
# MongoClient(f'mongodb://{host}:{port}/')
client = MongoClient(host, port)
client

MongoClient(host=['localhost:3306'], document_class=dict, tz_aware=False, connect=True)

In [23]:
# db = client['test-database']
db = client['test-database']
db

Database(MongoClient(host=['localhost:3306'], document_class=dict, tz_aware=False, connect=True), 'test-database')

In [24]:
# collection = db['test_collection']
collection = db.test_collection
collection

Collection(Database(MongoClient(host=['localhost:3306'], document_class=dict, tz_aware=False, connect=True), 'test-database'), 'test_collection')

In [ ]:
import datetime

post = {"author": "Mike",
        "text": "My first blog post!",
        "tags": ["mongodb", "python", "pymongo"],
        "date": datetime.datetime.utcnow()}

In [ ]:
post_id = db.test_collection.insert_one(post).inserted_id
post_id

In [ ]:
print('post_id: ', post_id)

print('db.list_collection_names(): ', db.list_collection_names())

print('db.test_collection.find_one(): ', db.test_collection.find_one())

print('db.test_collection.find_one({"author": "Mike"}): ', db.test_collection.find_one({"author": "Mike"}))
print('db.test_collection.find_one({"author": "Eliot"}): ', db.test_collection.find_one({"author": "Eliot"}))